# Chapter 16: Reinforcement Learning

## Exercises

### 1. How would you describe Reinforcement Learning? How is it different from regular supervised or unsupervised learning?

Reinforcement learning is the process of training a computer agent to complete make actions in an environment in order to earn rewards. The goal of RL is to train an agent policy which maximizes the reward the agent can accumulate. This differs from regular machine learning tasks which do not train an agent to take actions in an environment. 

### 2. Can you think of three possible applications of RL that were not mentioned in this chapter? For each of them, what is the environment? What is the agent? What are the possible actions? What are the rewards?

One application could be self-flying drones that are used to make deliveries. The environment is the physical world; the agent is the drone; the possible actions would be the different directions the drone could fly; and the rewards could be a function of the distance between the drone and the destination, if the drone was successfully able to reach the destination without hitting an obstacle, or the time it takes the drone to reach its destination.

Another application is training a model to learn to play a card game with fixed rules such as Blackjack. The environment would be a simulation of the card game. The agent would be a bot that plays the game. The possible actions would be two actions one can take while playing. The reward could be if the agent wins or loses the hand. You could also incorporate a betting round at the beginning of each hand and the goal could be trying to make as much money as possible.

A third application could be a cryptocurrency trading. The environment could be realtime prices of various cryptocurrencies. The agent could be a trading bot with access to cryptocurrency wallets for the different currencies. The The possible actions could be whether to buy or sell a particular amount of a certain cryptocurrency. The reward could be how much the agent's portfolio is worth in USD.



### 3. What is the discount rate? Can the optimal policy change if you modify the discount rate?

The discount rate is a constant (usually 0.95 or 0.99) that we multiply the rewards an agent gets from future actions. We do this so that the model slightly prefers short term rewards to rewards in the future. A different discount rate can cause a model to value long term reward more or less, which could change the policy the model learns.

### 4. How do you measure the performance of a Reinforcement Learning agent?

You measure the performance of an RL agent by either measuring how much reward the agent is able to accumulate while it performs the task. A more subjective way of measuring the performance is to observe the agent acting in its environment and see if you are satisfied with its performance.

### 5. What is the credit assignment problem? What does it occur? How can you alleviate it?

The credit assignment problem is the problem where an agent cannot determine what the long term effects of its immediate actions may be, so it cannot decide which next action will yield the most reward. You can alleviate the problem by training an agent using a Policy Gradients algorithm so it can learn what actions lead to later rewards.

### 6. What is the point of using a replay memory?

A replay memory stores an agent's past experience, e.g. an action it took, the state when it took the action, and what the resulting state was. During training, a model will randomly sample batches of memories from the replay memory, rather than only using its most recent experiences. The purpose of doing this is so that the agent does not correlate its policy too strongly to its most recent experiences.

### 7. What is an off-policy RL algorithm?

An off-policy RL algorithm is an algorithm that is different when it is being trained than whan it is executing the task it is being trained for. An example of an off-policy algorithm is the DQN trained to play Ms. PacMan in `MsPacMan.ipynb`.